In [1]:
%matplotlib inline
import time
import datetime
import cv2
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd

import sys
sys.path.append('../scripts')
import util

In [58]:
fname = '/home/trafferty/tmp/TTMLogStream_2017_05_17.csv'
f = open(fname, 'r')
ttm = f.readlines()
f.close()
len(ttm)

245430

In [22]:
cols = ['TS', 'C1 measx_mm','C1 scorex', 'C1 measy_mm','C1 scorey','C1 cmeasx_mm','C1 cscorex','C1 cmeasy_mm','C1 cscorey', 'C1 x unwrap cnt', 'C1 y unwrap cnt', 'C1 post measx_mm', 'C1 post measy_mm', 'C1 post cmeasx_mm', 'C1 post cmeasy_mm', 'C2 measx_mm','C2 scorex', 'C2 measy_mm','C2 scorey','C2 cmeasx_mm','C2 cscorex','C2 cmeasy_mm','C2 cscorey', 'C2 x unwrap cnt', 'C2 y unwrap cnt', 'C2 post measx_mm', 'C2 post measy_mm', 'C2 post cmeasx_mm', 'C2 post cmeasy_mm', 'C3 measx_mm','C3 scorex', 'C3 measy_mm','C3 scorey','C3 cmeasx_mm','C3 cscorex','C3 cmeasy_mm','C3 cscorey', 'C3 x unwrap cnt', 'C3 y unwrap cnt', 'C3 post measx_mm', 'C3 post measy_mm', 'C3 post cmeasx_mm', 'C3 post cmeasy_mm', 'C4 measx_mm','C4 scorex', 'C4 measy_mm', 'C4 scorey', 'C4 cmeasx_mm', 'C4 cscorex', 'C4 cmeasy_mm', 'C4 cscorey', 'C4 x unwrap cnt', 'C4 y unwrap cnt', 'C4 post measx_mm', 'C4 post measy_mm', 'C4 post cmeasx_mm', 'C4 post cmeasy_mm']
print(len(cols))

57


In [ ]:
def parse_into_data_sets(data):
    data_ranges = []
    max_delta = 1
    start = 0
    for idx, d in enumerate(data):
        if idx == 0:
            d_prev = d
        else:
            ts1 = d_prev.split(',')[0][-23:]
            ts2 = d.split(',')[0][-23:]
            dt1 = datetime.datetime.strptime(ts1, "%Y-%m-%d %H:%M:%S.%f")
            dt2 = datetime.datetime.strptime(ts2, "%Y-%m-%d %H:%M:%S.%f")
            s1 =time.mktime(dt1.timetuple()) + (dt1.microsecond / 1000000.0)
            s2 =time.mktime(dt2.timetuple()) + (dt2.microsecond / 1000000.0)
            #delta = float(d.split(',')[0][-6:]) - float(d_prev.split(',')[0][-6:])
            delta = s2 - s1
            if delta > max_delta:
                end = idx - 1
                data_ranges.append((start, end))
                start = idx
            d_prev = d
    data_ranges.append((start, idx))
    len(data_ranges)

    data_sets = []
    for idx, data_range in enumerate(data_ranges):
        data_sets.append(data[data_range[0]:data_range[1]])
        print("Data set %d, range: %d - %d (total: %d)" % (idx, data_range[0], data_range[1], (data_range[1]-data_range[0])))
        
    for idx, ds in enumerate(data_sets):
        print("[%d] range: %s -> %s" % ( idx, ds[0].split(',')[0], ds[-1].split(',')[0]))
        
    return data_ranges, data_sets

def cleanup_and_format_data_set(ds):
    ds_new = []
    for idx, d in enumerate(ds):
        dnew_lst = []
        d_lst = d.split(',')
        #print(idx)
        dnew_lst.append(d_lst[0][17:])
        dnew_lst.append(float(d_lst[2]))                 
        dnew_lst.append(int(d_lst[3]))                   
        dnew_lst.append(float(d_lst[4]))                 
        dnew_lst.append(int(d_lst[5]))                   
        dnew_lst.append(float(d_lst[6]))                 
        dnew_lst.append(int(d_lst[7]))
        dnew_lst.append(float(d_lst[8]))                 
        dnew_lst.append(int(d_lst[9]))                   
        dnew_lst.append((d_lst[10].count('0') - 1) * -1)
        dnew_lst.append((d_lst[11].count('0') - 1) * -1)
        dnew_lst.append(float(d_lst[13]))                
        dnew_lst.append(float(d_lst[14]))                
        dnew_lst.append(float(d_lst[15]))                
        dnew_lst.append(float(d_lst[16]))                
        dnew_lst.append(float(d_lst[18]))                
        dnew_lst.append(int(d_lst[19]))                  
        dnew_lst.append(float(d_lst[20]))                
        dnew_lst.append(int(d_lst[21]))                  
        dnew_lst.append(float(d_lst[22]))                
        dnew_lst.append(int(d_lst[23]))                  
        dnew_lst.append(float(d_lst[24]))                
        dnew_lst.append(int(d_lst[25]))                  
        dnew_lst.append((d_lst[26].count('0') - 1) * -1)
        dnew_lst.append((d_lst[27].count('0') - 1) * -1)
        dnew_lst.append(float(d_lst[29]))                
        dnew_lst.append(float(d_lst[30]))                
        dnew_lst.append(float(d_lst[31]))                
        dnew_lst.append(float(d_lst[32]))                
        dnew_lst.append(float(d_lst[34]))                
        dnew_lst.append(int(d_lst[35]))                  
        dnew_lst.append(float(d_lst[36]))                
        dnew_lst.append(int(d_lst[37]))                  
        dnew_lst.append(float(d_lst[38]))                
        dnew_lst.append(int(d_lst[39]))                  
        dnew_lst.append(float(d_lst[40]))                
        dnew_lst.append(int(d_lst[41]))                  
        dnew_lst.append((d_lst[42].count('0') - 1) * -1)
        dnew_lst.append((d_lst[43].count('0') - 1) * -1)
        dnew_lst.append(float(d_lst[45]))                
        dnew_lst.append(float(d_lst[46]))                
        dnew_lst.append(float(d_lst[47]))                
        dnew_lst.append(float(d_lst[48]))                
        dnew_lst.append(float(d_lst[50]))                
        dnew_lst.append(int(d_lst[51]))                  
        dnew_lst.append(float(d_lst[52]))                
        dnew_lst.append(int(d_lst[53]))                  
        dnew_lst.append(float(d_lst[54]))                
        dnew_lst.append(int(d_lst[55]))                  
        dnew_lst.append(float(d_lst[56]))                
        dnew_lst.append(int(d_lst[57]))                  
        dnew_lst.append((d_lst[58].count('0') - 1) * -1)
        dnew_lst.append((d_lst[59].count('0') - 1) * -1)
        dnew_lst.append(float(d_lst[61]))                
        dnew_lst.append(float(d_lst[62]))                
        dnew_lst.append(float(d_lst[63]))                
        dnew_lst.append(float(d_lst[64]))
        ds_new.append(dnew_lst)
    return ds_new
        
        

In [80]:
data_ranges, data_sets_rough = parse_into_data_sets(ttm)

Data set 0, range: 0 - 11708 (total: 11708)
Data set 1, range: 11709 - 23305 (total: 11596)
Data set 2, range: 23306 - 35013 (total: 11707)
Data set 3, range: 35014 - 46612 (total: 11598)
Data set 4, range: 46613 - 58322 (total: 11709)
Data set 5, range: 58323 - 69917 (total: 11594)
Data set 6, range: 69918 - 81626 (total: 11708)
Data set 7, range: 81627 - 92889 (total: 11262)
Data set 8, range: 92890 - 104602 (total: 11712)
Data set 9, range: 104603 - 116312 (total: 11709)
Data set 10, range: 116313 - 127916 (total: 11603)
Data set 11, range: 127917 - 139627 (total: 11710)
Data set 12, range: 139628 - 151222 (total: 11594)
Data set 13, range: 151223 - 162924 (total: 11701)
Data set 14, range: 162925 - 174627 (total: 11702)
Data set 15, range: 174628 - 175947 (total: 1319)
Data set 16, range: 175948 - 187546 (total: 11598)
Data set 17, range: 187547 - 198925 (total: 11378)
Data set 18, range: 198926 - 210526 (total: 11600)
Data set 19, range: 210527 - 222238 (total: 11711)
Data set 20,

In [83]:
c = {}
for idx, ds in enumerate(data_sets_rough):
    for d in ds:
        cnt = d.count(',')
        if cnt not in c:
            c[cnt] = 1
        else:
            c[cnt] += 1
print(c)

{64: 245408}


In [ ]:
data_sets = []
for idx, ds in enumerate(data_sets_rough):
    data_sets.append(cleanup_and_format_data_set(ds))

In [82]:
c = {}
for idx, ds in enumerate(data_sets):
    for d in ds:
        #cnt = d.count(',')
        cnt = len(d)
        if cnt not in c:
            c[cnt] = 1
        else:
            c[cnt] += 1
print(c)

{57: 245408}


In [84]:
data_sets[0][0]

['2017-05-17 08:57:01.341',
 -0.0001886,
 0,
 0.000139,
 0,
 0.000884,
 0,
 -0.000959,
 0,
 1,
 1,
 0.0008114,
 -0.000861,
 0.000884,
 -0.000959,
 0.001,
 1,
 0.001,
 1,
 0.0,
 1,
 0.0,
 1,
 -1,
 -1,
 nan,
 nan,
 0.0,
 0.0,
 0.001,
 1,
 0.001,
 1,
 0.0,
 1,
 0.0,
 1,
 -1,
 -1,
 nan,
 nan,
 0.0,
 0.0,
 0.0005164,
 0,
 -0.0004502,
 0,
 -0.001022,
 0,
 -0.001231,
 0,
 1,
 1,
 -0.0014836,
 -0.0014502,
 -0.001022,
 -0.001231]